# Задание 2

- Загрузите файл HR.csv в pandas dataframe.
- Рассчитайте основные статистики для переменных: среднее, медиана, мода, мин/макс, среднее отклонение.
- Рассчитайте и визуализируйте корреляционную матрицу для количественных переменных. Определите две самые скоррелированные и две наименее скоррелированные переменные.
- Рассчитайте, сколько сотрудников работает в каждом департаменте.
- Покажите распределение сотрудников по зарплатам.
- Покажите распределение сотрудников по зарплатам в каждом департаменте по отдельности.
- Проверьте гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом.
- Рассчитайте следующие показатели среди уволившихся и не уволившихся сотрудников (по отдельности):
  - доля сотрудников с повышением за последние 5 лет;
  - средняя степень удовлетворённости;
  - среднее количество проектов.
- Разделите данные на тестовую и обучающую выборки.
- Постройте модель LDA, которая показывает, уволился ли сотрудник на основе имеющихся факторов (кроме department и salary).
- Оцените качество модели на тестовой выборке. Используйте метод accuracy.
- Загрузите Jupyter Notebook с решением на GitHub и разместите ссылку на ваше решение в личном кабинете в «Итоговом задании. Часть 2».

In [16]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
from scipy import stats
import statistics
import random
import plotly.figure_factory as ff

## 1. Загрузите файл HR.csv в pandas dataframe.

In [9]:
df = pd.read_csv('HR.csv')

In [10]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## 2. Рассчитайте основные статистики для переменных: среднее, медиана, мода, мин/макс, среднее отклонение.

In [11]:
df.iloc[:,:5].describe().round(2).loc[['50%', 'mean', 'min', 'max', 'std']]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company
50%,0.64,0.72,4.00,200.00,3.00
mean,0.61,0.72,3.80,201.05,3.50
min,0.09,0.36,2.00,96.00,2.00
max,1.00,1.00,7.00,310.00,10.00
std,0.25,0.17,1.23,49.94,1.46


## 3. Рассчитайте и визуализируйте корреляционную матрицу для количественных переменных. Определите две самые скоррелированные и две наименее скоррелированные переменные.

In [12]:
df.iloc[:,:5].corr().round(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company
satisfaction_level,1.000,0.105,-0.143,-0.020,-0.101
last_evaluation,0.105,1.000,0.349,0.340,0.132
number_project,-0.143,0.349,1.000,0.417,0.197
average_montly_hours,-0.020,0.340,0.417,1.000,0.128
time_spend_company,-0.101,0.132,0.197,0.128,1.000


In [19]:
df_corrs = df[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']].corr()

fig = ff.create_annotated_heatmap(z=df_corrs.values, x=list(df_corrs.index), 
                                  y=list(df_corrs.columns), 
                                  annotation_text=df_corrs.round(3).values, 
                                  showscale=True, colorscale='Cividis')
fig.show()

- Наиболее скореллированные переменные: average_montly_hours и number_project - 0.417
- Наименее скоррелированные переменные average_montly_hours и satisfaction_level - (-0.02)

## 4. Рассчитайте, сколько сотрудников работает в каждом департаменте.

In [20]:
df.groupby('department').size()

department
IT             1227
RandD           787
accounting      767
hr              739
management      630
marketing       858
product_mng     902
sales          4140
support        2229
technical      2720
dtype: int64

## 5. Покажите распределение сотрудников по зарплатам.

In [21]:
df.salary.value_counts()

low       7316
medium    6446
high      1237
Name: salary, dtype: int64

## 6. Покажите распределение сотрудников по зарплатам в каждом департаменте по отдельности.

In [22]:
df.pivot_table(index='department', columns='salary', aggfunc='size')

salary,high,low,medium
department,,,
IT,83,609,535
RandD,51,364,372
accounting,74,358,335
hr,45,335,359
management,225,180,225
marketing,80,402,376
product_mng,68,451,383
sales,269,2099,1772
support,141,1146,942


## 7. Проверьте гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом.

In [23]:
print('Среднее количество рабочих часов сотрудников c низким окладом', df[df.salary == 'low']['average_montly_hours'].mean())
print('Среднее количество рабочих часов сотрудников с высоким окладом', df[df.salary == 'high']['average_montly_hours'].mean())

Среднее количество рабочих часов сотрудников c низким окладом 200.9965828321487
Среднее количество рабочих часов сотрудников с высоким окладом 199.86742118027485


Гипотеза

H0 - в среднем проведенное на работе время одинаково для всех работников

In [24]:
salary_low = df[df['salary'] == 'low']['average_montly_hours']
salary_high = df[df['salary'] == 'high']['average_montly_hours']

In [25]:
result = stats.ttest_ind(salary_low, salary_high, equal_var=False)

a = 0.05

if (result.pvalue < a):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

Не отвергаем нулевую гипотезу


В среднем проведённое на работе время одинаково для всех работников.

## 8. Рассчитайте следующие показатели среди уволившихся и не уволившихся сотрудников (по отдельности):
  - доля сотрудников с повышением за последние 5 лет;
  - средняя степень удовлетворённости;
  - среднее количество проектов.

In [26]:
workers = df[(df['left'] == 0)]
retireds =  df[(df['left'] == 1)]

In [27]:
indicators = pd.DataFrame()
rows = []

rows.append(
    {
    'Показатель': 'Доля получивших повышение',
    'Уволившиеся': round(retireds.promotion_last_5years.value_counts(normalize=True)[1], 3),
    'Действующие': round(workers.promotion_last_5years.value_counts(normalize=True)[1], 3)
    }
)

rows.append(
    {
    'Показатель': 'Средняя степень удовлетворенности',
    'Уволившиеся': round(retireds.satisfaction_level.mean(), 3),
    'Действующие': round(workers.satisfaction_level.mean(), 3)
    }
)

rows.append(
    {
    'Показатель': 'Среднее количество проектов',
    'Уволившиеся': round(retireds.number_project.mean(), 3),
    'Действующие': round(workers.number_project.mean(), 3)
    }
)

for row in rows:
    indicators = pd.concat([indicators, pd.DataFrame([row])])

indicators

,Показатель,Уволившиеся,Действующие
0,Доля получивших повышение,0.005,0.026
0,Средняя степень удовлетворенности,0.440,0.667
0,Среднее количество проектов,3.856,3.787


## 9. Разделите данные на тестовую и обучающую выборки.

In [28]:
X = df.iloc[:,[0,1,2,3,4,5,7]]
y = df['left']

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## 10. Постройте модель LDA, которая показывает, уволился ли сотрудник на основе имеющихся факторов (кроме department и salary).

In [33]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [34]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [41]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [42]:
model.score(X_test, y_test).round(2)

0.76

## 11. Оцените качество модели на тестовой выборке. Используйте метод accuracy.

In [43]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(y_test, model.predict(X_test)).round(2)

0.76